<a href="https://colab.research.google.com/github/chunweixu/repo/blob/main/content/2.%20%E6%8F%90%E7%A4%BA%E5%8E%9F%E5%88%99%20Guidelines.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 吴恩达老师的Prompt Engineering课程学习

本项目由李鲁鲁老师在DataWahle翻译的中文版本的吴恩达老师与 OpenAI 合作推出的 《ChatGPT Prompt Engineering for Developers》基础上，进行了一些自己的实践思考。

原来项目的链接是 [https://github.com/datawhalechina/prompt-engineering-for-developers](https://github.com/datawhalechina/prompt-engineering-for-developers)

每一课都在colab上进行了改造，并将链接放在了readme，可以直接用colab进行学习。

# 第二章 编写 Prompt 的原则

  本章的主要内容为编写 Prompt 的原则，在本章中，我们将给出两个编写 Prompt 的原则与一些相关的策略，你将练习基于这两个原则来编写有效的 Prompt，从而便捷而有效地使用 LLM。

## 一、环境配置

本教程使用 OpenAI 所开放的 ChatGPT API，因此你需要首先拥有一个 ChatGPT 的 API_KEY（也可以直接访问官方网址在线测试），然后需要安装 openai 的第三方库

首先需要安装所需第三方库：

openai：

```bash
pip install openai
```

dotenv:

```bash
pip install -U python-dotenv
```

In [ ]:
! pip install openai

In [ ]:
! pip install python-dotenv

In [ ]:
# 将自己的 API-KEY 导入系统环境变量
# !export OPENAI_API_KEY='api-key'
# 李鲁鲁: colab 不需要走这一步

In [ ]:
import openai
import os
from dotenv import load_dotenv, find_dotenv
# 导入第三方库

# _ = load_dotenv(find_dotenv())
# 读取系统中的环境变量

openai.api_key  = 'sk-'
# 李鲁鲁 在这里设置你的API_KEY

我们将在后续课程中深入探究 OpenAI 提供的 ChatCompletion API 的使用方法，在此处，我们先将它封装成一个函数，你无需知道其内部机理，仅需知道调用该函数输入 Prompt 其将会给出对应的 Completion 即可。

In [ ]:
# 一个封装 OpenAI 接口的函数，参数为 Prompt，返回对应结果
def get_completion(prompt, model="gpt-3.5-turbo"):
    '''
    prompt: 对应的提示
    model: 调用的模型，默认为 gpt-3.5-turbo(ChatGPT)，有内测资格的用户可以选择 gpt-4
    '''
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # 模型输出的温度系数，控制输出的随机程度
    )
    # 调用 OpenAI 的 ChatCompletion 接口
    return response.choices[0].message["content"]


## 二、两个基本原则

### 原则一：编写清晰、具体的指令

你应该通过提供尽可能清晰和具体的指令来表达您希望模型执行的操作。这将引导模型给出正确的输出，并减少你得到无关或不正确响应的可能。编写清晰的指令不意味着简短的指令，因为在许多情况下，更长的提示实际上更清晰且提供了更多上下文，这实际上可能导致更详细更相关的输出。

**策略一：使用分隔符清晰地表示输入的不同部分**，分隔符可以是：```，""，<>，\<tag>，<\tag>等

你可以使用任何明显的标点符号将特定的文本部分与提示的其余部分分开。这可以是任何可以使模型明确知道这是一个单独部分的标记。使用分隔符是一种可以避免提示注入的有用技术。提示注入是指如果用户将某些输入添加到提示中，则可能会向模型提供与您想要执行的操作相冲突的指令，从而使其遵循冲突的指令而不是执行您想要的操作。即，输入里面可能包含其他指令，会覆盖掉你的指令。对此，使用分隔符是一个不错的策略。

以下是一个例子，我们给出一段话并要求 GPT 进行总结，在该示例中我们使用 ``` 来作为分隔符


In [ ]:
# 中文版见下一个 cell
text = f"""
You should express what you want a model to do by \ 
providing instructions that are as clear and \ 
specific as you can possibly make them. \ 
This will guide the model towards the desired output, \ 
and reduce the chances of receiving irrelevant \ 
or incorrect responses. Don't confuse writing a \ 
clear prompt with writing a short prompt. \ 
In many cases, longer prompts provide more clarity \ 
and context for the model, which can lead to \ 
more detailed and relevant outputs.
"""
prompt = f"""
Summarize the text delimited by triple backticks \ 
into a single sentence.
```{text}```
"""
response = get_completion(prompt)
print(response)

Clear and specific instructions should be provided to guide a model towards the desired output, and longer prompts can provide more clarity and context for the model, leading to more detailed and relevant outputs.


李鲁鲁.其实这里如果你觉得看英文累，也可以在prompt最后增加"then translate the result into Chinese".

In [ ]:
text = f"""
你应该提供尽可能清晰、具体的指示，以表达你希望模型执行的任务。\
这将引导模型朝向所需的输出，并降低收到无关或不正确响应的可能性。\
不要将写清晰的提示与写简短的提示混淆。\
在许多情况下，更长的提示可以为模型提供更多的清晰度和上下文信息，从而导致更详细和相关的输出。
"""
# 需要总结的文本内容
prompt = f"""
把用三个反引号括起来的文本总结成一句话。
```{text}```
"""
# 指令内容，使用 ``` 来分隔指令和待总结的内容
response = get_completion(prompt)
print(response)

提供清晰具体的指示，避免无关或不正确响应，不要混淆写清晰和写简短，更长的提示可以提供更多清晰度和上下文信息，导致更详细和相关的输出。


李鲁鲁：反引号没有那么符合中文习惯，我一般使用三个双引号

**策略二：要求一个结构化的输出**，可以是 Json、HTML 等格式

第二个策略是要求生成一个结构化的输出，这可以使模型的输出更容易被我们解析，例如，你可以在 Python 中将其读入字典或列表中。。

在以下示例中，我们要求 GPT 生成三本书的标题、作者和类别，并要求 GPT 以 Json 的格式返回给我们，为便于解析，我们指定了 Json 的键。

In [ ]:
prompt = f"""
Generate a list of three made-up book titles along \ 
with their authors and genres. 
Provide them in JSON format with the following keys: 
book_id, title, author, genre.
"""
response = get_completion(prompt)
print(response)


[
  {
    "book_id": 1,
    "title": "The Lost City of Zorath",
    "author": "Aria Blackwood",
    "genre": "Fantasy"
  },
  {
    "book_id": 2,
    "title": "The Last Survivors",
    "author": "Ethan Stone",
    "genre": "Science Fiction"
  },
  {
    "book_id": 3,
    "title": "The Secret Life of Bees",
    "author": "Lila Rose",
    "genre": "Romance"
  }
]


In [ ]:
prompt = f"""
请生成包括书名、作者和类别的三本虚构书籍清单，\
并以 JSON 格式提供，其中包含以下键:book_id、title、author、genre。
"""
response = get_completion(prompt)
print(response)


{
  "books": [
    {
      "book_id": 1,
      "title": "The Shadow of the Wind",
      "author": "Carlos Ruiz Zafón",
      "genre": "Mystery"
    },
    {
      "book_id": 2,
      "title": "The Name of the Wind",
      "author": "Patrick Rothfuss",
      "genre": "Fantasy"
    },
    {
      "book_id": 3,
      "title": "The Hitchhiker's Guide to the Galaxy",
      "author": "Douglas Adams",
      "genre": "Science Fiction"
    }
  ]
}


李鲁鲁：这里不知道DataWhale有没有发现，内容仍然是英文的。

我们显然没有那么希望是这样的 我们还是希望书的标题、作者是中文，让我们来试试下面这个prompt

In [ ]:
prompt = f"""
请生成包括书名、作者和类别的三本虚构书籍清单，其中标题和作者需要是中文的\
并以 JSON 格式提供，其中包含以下键:标题、作者、book_id、genre。
"""
response = get_completion(prompt)
print(response)


{
  "book1": {
    "标题": "时间之河",
    "作者": "张三",
    "book_id": "001",
    "genre": "科幻"
  },
  "book2": {
    "标题": "红楼梦",
    "作者": "曹雪芹",
    "book_id": "002",
    "genre": "古典文学"
  },
  "book3": {
    "标题": "三体",
    "作者": "刘慈欣",
    "book_id": "003",
    "genre": "科幻"
  }
}


**策略三：要求模型检查是否满足条件**

如果任务做出的假设不一定满足，我们可以告诉模型先检查这些假设，如果不满足，指示并停止执行。你还可以考虑潜在的边缘情况以及模型应该如何处理它们，以避免意外的错误或结果。

在如下示例中，我们将分别给模型两段文本，分别是制作茶的步骤以及一段没有明确步骤的文本。我们将要求模型判断其是否包含一系列指令，如果包含则按照给定格式重新编写指令，不包含则回答未提供步骤。

In [ ]:
text_1 = f"""
Making a cup of tea is easy! First, you need to get some \ 
water boiling. While that's happening, \ 
grab a cup and put a tea bag in it. Once the water is \ 
hot enough, just pour it over the tea bag. \ 
Let it sit for a bit so the tea can steep. After a \ 
few minutes, take out the tea bag. If you \ 
like, you can add some sugar or milk to taste. \ 
And that's it! You've got yourself a delicious \ 
cup of tea to enjoy.
"""
prompt = f"""
You will be provided with text delimited by triple quotes. 
If it contains a sequence of instructions, \ 
re-write those instructions in the following format:

Step 1 - ...
Step 2 - …
…
Step N - …

If the text does not contain a sequence of instructions, \ 
then simply write \"No steps provided.\"

\"\"\"{text_1}\"\"\"
"""
response = get_completion(prompt)
print("Completion for Text 1:")
print(response)

Completion for Text 1:
Step 1 - Get some water boiling.
Step 2 - Grab a cup and put a tea bag in it.
Step 3 - Once the water is hot enough, pour it over the tea bag.
Step 4 - Let it sit for a bit so the tea can steep.
Step 5 - After a few minutes, take out the tea bag.
Step 6 - Add some sugar or milk to taste.
Step 7 - Enjoy your delicious cup of tea!




In [ ]:
text_2 = f"""
The sun is shining brightly today, and the birds are \
singing. It's a beautiful day to go for a \ 
walk in the park. The flowers are blooming, and the \ 
trees are swaying gently in the breeze. People \ 
are out and about, enjoying the lovely weather. \ 
Some are having picnics, while others are playing \ 
games or simply relaxing on the grass. It's a \ 
perfect day to spend time outdoors and appreciate the \ 
beauty of nature.
"""
prompt = f"""You will be provided with text delimited by triple quotes. 
If it contains a sequence of instructions, \ 
re-write those instructions in the following format:
Step 1 - ...
Step 2 - …
…
Step N - …

If the text does not contain a sequence of instructions, \ 
then simply write \"No steps provided.\"

\"\"\"{text_2}\"\"\"
"""
response = get_completion(prompt)
print("Completion for Text 2:")
print(response)

Completion for Text 2:
No steps provided.


李鲁鲁：我觉得这里特别好的是这个prompt自带一个garding，对于预料外的输入，可以有很好的处理，一会儿我来试验一个真实的我们需要的例子。

In [ ]:
# 有步骤的文本
text_1 = f"""
泡一杯茶很容易。首先，需要把水烧开。\
在等待期间，拿一个杯子并把茶包放进去。\
一旦水足够热，就把它倒在茶包上。\
等待一会儿，让茶叶浸泡。几分钟后，取出茶包。\
如果你愿意，可以加一些糖或牛奶调味。\
就这样，你可以享受一杯美味的茶了。
"""
prompt = f"""
您将获得由三个引号括起来的文本。\
如果它包含一系列的指令，则需要按照以下格式重新编写这些指令：

第一步 - ...
第二步 - …
…
第N步 - …

如果文本中不包含一系列的指令，则直接写“未提供步骤”。"
\"\"\"{text_1}\"\"\"
"""
response = get_completion(prompt)
print("Text 1 的总结:")
print(response)

Text 1 的总结:
第一步 - 把水烧开。
第二步 - 拿一个杯子并把茶包放进去。
第三步 - 把热水倒在茶包上。
第四步 - 等待几分钟，让茶叶浸泡。
第五步 - 取出茶包。
第六步 - 如果你愿意，可以加一些糖或牛奶调味。
第七步 - 就这样，你可以享受一杯美味的茶了。


In [ ]:
# 无步骤的文本
text_2 = f"""
今天阳光明媚，鸟儿在歌唱。\
这是一个去公园散步的美好日子。\
鲜花盛开，树枝在微风中轻轻摇曳。\
人们外出享受着这美好的天气，有些人在野餐，有些人在玩游戏或者在草地上放松。\
这是一个完美的日子，可以在户外度过并欣赏大自然的美景。
"""
prompt = f"""
您将获得由三个引号括起来的文本。\
如果它包含一系列的指令，则需要按照以下格式重新编写这些指令：

第一步 - ...
第二步 - …
…
第N步 - …

如果文本中不包含一系列的指令，则直接写“未提供步骤”。"
\"\"\"{text_2}\"\"\"
"""
response = get_completion(prompt)
print("Text 2 的总结:")
print(response)

Text 2 的总结:
未提供步骤。


李鲁鲁: 让我们来试一个我们在骆驼大乱斗中想完成的事情

就是总结小说中的斗法片段

In [ ]:
# 有步骤的文本
text_1 = f"""
韩立叹了一口气，原本还想费些口舌，看看能否糊弄过去，但如今看对方的神态和凭对方的心计，是一点迂回的余地都没有了，他和自己肯定只能有一个还可活在世上，还是不要白费口舌，先下手为强算了。\
想到这里，韩立二话不说，一扬左手一个精钢环就发出怪啸之声，直向“陆师兄”冲去，随后又把右手一亮，青黑色的葫芦出现在了手中，并从葫芦嘴中喷出了五六个黑糊糊的圆球，紧跟在钢环后而去。\
做完这一切韩立并未罢手，他空出的左手在虚空中略一比划，刹那间浮现了数个红色火球，袖子略微一动，把这些火球卷在其中，紧接着再冲“陆师兄”猛然一甩，口中低吐一个“去”字。\
顿时，夹带着一股炎热之气，火球们一窝蜂的四行散开，不同角度砸向了陆师兄。\
这一番出手，韩立几乎动用了未得到新法器之前，不使用符宝的一切进攻手段，特别是最后这手数弹并发的瞬发手法，更是韩立费了好大工夫才从吴风那里学到手的，为的就是想要打对方一个措手不及，一举闪电击毙对手。
"""
prompt = f"""
您将获得由三个引号括起来的文本。\
如果它包含一系列的打斗的片段，则需要按照以下回合制描述的格式重新编写这些片段：

第一回合 - <A> 使用 <A's skill>, <skill result description>
第二回合 - <B> 使用 <B's skill>, <skill result description>
…
第N回合 - …

如果文本中不包含一系列的打斗，则直接写“未提供打斗细节”。"
\"\"\"{text_1}\"\"\"
"""
response = get_completion(prompt)
print("Text 1 的总结:")
print(response)

Text 1 的总结:
第一回合 - 韩立 使用 精钢环, 击中“陆师兄”
第二回合 - 韩立 使用 葫芦中的黑糊糊圆球, 击中“陆师兄”
第三回合 - 韩立 使用 火球术, 击中“陆师兄”
未提供打斗细节。


**策略四：提供少量示例**

即在要求模型执行实际任务之前，提供给它少量成功执行任务的示例。

例如，在以下的示例中，我们告诉模型其任务是以一致的风格回答问题，并先给它一个孩子和一个祖父之间的对话的例子。孩子说，“教我耐心”，祖父用这些隐喻回答。因此，由于我们已经告诉模型要以一致的语气回答，现在我们说“教我韧性”，由于模型已经有了这个少样本示例，它将以类似的语气回答下一个任务。

In [ ]:
prompt = f"""
Your task is to answer in a consistent style.

<child>: Teach me about patience.

<grandparent>: The river that carves the deepest \ 
valley flows from a modest spring; the \ 
grandest symphony originates from a single note; \ 
the most intricate tapestry begins with a solitary thread.

<child>: Teach me about resilience.
"""
response = get_completion(prompt)
print(response)

<grandparent>: Resilience is like a tree that bends with the wind but never breaks. It is the ability to bounce back from adversity and keep moving forward, even when things get tough. Just like a tree that grows stronger with each storm it weathers, resilience is a quality that can be developed and strengthened over time.


In [ ]:
prompt = f"""
你的任务是以一致的风格回答问题。

<孩子>: 教我耐心。

<祖父母>: 挖出最深峡谷的河流源于一处不起眼的泉眼；最宏伟的交响乐从单一的音符开始；最复杂的挂毯以一根孤独的线开始编织。

<孩子>: 教我韧性。
"""
response = get_completion(prompt)
print(response)

<祖父母>: 韧性就像是一棵树，它需要经历风吹雨打、寒冬酷暑，才能成长得更加坚强。所以，当你遇到挫折和困难时，不要轻易放弃，要像树一样坚定地扎根，不断成长，最终成为一棵高大的树。


李鲁鲁：这种说教风格的学习

直接让我想起了我小时候中二阶段（国中二年级）的文学创作巅峰

那个时候很喜欢 纪伯伦的 《先知》 觉得先知的说法 风格非常说教

我们不如现学现卖一段，这里就不使用代码了。


这里我们想让 《先知》 故事中的先知

来讨论时间这个概念， 于是ChatGPT的输入

```
你的任务是以一致的风格回答问题。

<市民>: 请和我们讨论一下"孩子"

<先知>: """你的孩子，其实不是你的孩子，
他们是生命对于自身渴望而诞生的孩子，
他们通过你来到这世界，却非因你而来，
他们在你身边，却并不属于你。
你可以给予他们的是你的爱，却不是你的想法，
因为他们自己有自己的思想。
你可以庇护的是他们的身体，却不是他们的灵魂，
因为他们的灵魂属于明天，属于你做梦也无法达到的明天。
你可以拼尽全力，变得象他们一样，
却不要让他们变得和你一样，
因为生命不会后退，也不在过去停留。
你是弓，儿女是从你那里射出的箭。
弓箭手望着未来之路上的箭靶，
他用尽力气将你拉开，使他的箭射得又快又远。
怀着快乐的心情，在弓箭手的手里弯曲吧，
因为他爱一路飞翔的箭，也爱无比稳定的弓。"""

<市民>:请和我们讨论一下"衣服"

<先知>: 你们的衣服遮掩了你们许多的美，却不能遮盖住丑。
尽管你们借衣服寻求隐私的自由，但你们找到的却是羁绊和束缚。
但愿你们用自己的肌肤而不是衣服去迎接阳光和清风，
因为阳光中有生命的气息，而风中有生命之手。
你们中一些人说：“是北风织造了我们所穿的衣服。”
我说，是的，的确是北风，
但它以羞怯为织机，以纤弱的肌肉为纱线。
它一旦完成工作，便会在林中大笑。
不要忘记，羞怯原是抵挡不洁目光的盾牌。
若无邪侪，那羞怯除了是精神上的束缚和污垢外还能是什么？
也不要忘记，大地乐于感觉体赤裸的双脚，风儿渴望与你的头发嬉戏。

<市民>:请和我们讨论一下"时间"
```

让我们来看看GPT的输出

```
<先知>: 时间是一条河流，它汇聚着每个生命的流动和消逝。
每个人都在时间的河流中漂泊，不断地向前推进。
我们无法停止或掌控时间，但可以选择如何利用它。
时间是宝贵的，它可以让我们成长、学习、改变和创造。
但若我们不珍惜时间，它也会成为我们生命中的遗憾和遗失。
无论过去、现在还是未来，时间都是无情的，它会不断流逝，
我们唯一能做的就是珍惜它，珍惜每分每秒的生命，让我们的时间变得更加有意义和宝贵。
```

这个回答可以说是相当不错了。

这里我们来挖个坑吧，我收集了所有纪伯伦的诗，如果这套课刷完了

可以做个自动哲理机器人。。。

### 原则二：给模型时间去思考

如果模型匆忙地得出了错误的结论，您应该尝试重新构思查询，请求模型在提供最终答案之前进行一系列相关的推理。换句话说，如果您给模型一个在短时间或用少量文字无法完成的任务，它可能会猜测错误。这种情况对人来说也是一样的。如果您让某人在没有时间计算出答案的情况下完成复杂的数学问题，他们也可能会犯错误。因此，在这些情况下，您可以指示模型花更多时间思考问题，这意味着它在任务上花费了更多的计算资源。

**策略一：指定完成任务所需的步骤**

接下来我们将通过给定一个复杂任务，给出完成该任务的一系列步骤，来展示这一策略的效果

首先我们描述了杰克和吉尔的故事，并给出一个指令。该指令是执行以下操作。首先，用一句话概括三个反引号限定的文本。第二，将摘要翻译成法语。第三，在法语摘要中列出每个名称。第四，输出包含以下键的 JSON 对象：法语摘要和名称数。然后我们要用换行符分隔答案。

In [ ]:
text = f"""
In a charming village, siblings Jack and Jill set out on \ 
a quest to fetch water from a hilltop \ 
well. As they climbed, singing joyfully, misfortune \ 
struck—Jack tripped on a stone and tumbled \ 
down the hill, with Jill following suit. \ 
Though slightly battered, the pair returned home to \ 
comforting embraces. Despite the mishap, \ 
their adventurous spirits remained undimmed, and they \ 
continued exploring with delight.
"""
# example 1
prompt_1 = f"""
Perform the following actions: 
1 - Summarize the following text delimited by triple \
backticks with 1 sentence.
2 - Translate the summary into French.
3 - List each name in the French summary.
4 - Output a json object that contains the following \
keys: french_summary, num_names.

Separate your answers with line breaks.

Text:
```{text}```
"""
response = get_completion(prompt_1)
print("Completion for prompt 1:")
print(response)

Completion for prompt 1:
Two siblings, Jack and Jill, go on a quest to fetch water from a well on a hilltop, but misfortune strikes and they both tumble down the hill, returning home slightly battered but with their adventurous spirits undimmed.

Deux frères et sœurs, Jack et Jill, partent en quête d'eau d'un puits sur une colline, mais un malheur frappe et ils tombent tous les deux de la colline, rentrant chez eux légèrement meurtris mais avec leurs esprits aventureux intacts. 
Noms: Jack, Jill.

{
  "french_summary": "Deux frères et sœurs, Jack et Jill, partent en quête d'eau d'un puits sur une colline, mais un malheur frappe et ils tombent tous les deux de la colline, rentrant chez eux légèrement meurtris mais avec leurs esprits aventureux intacts.",
  "num_names": 2
}


In [ ]:
text = f"""
在一个迷人的村庄里，兄妹杰克和吉尔出发去一个山顶井里打水。\
他们一边唱着欢乐的歌，一边往上爬，\
然而不幸降临——杰克绊了一块石头，从山上滚了下来，吉尔紧随其后。\
虽然略有些摔伤，但他们还是回到了温馨的家中。\
尽管出了这样的意外，他们的冒险精神依然没有减弱，继续充满愉悦地探索。
"""
# example 1
prompt_1 = f"""
执行以下操作：
1-用一句话概括下面用三个反引号括起来的文本。
2-将摘要翻译成法语。
3-在法语摘要中列出每个人名。
4-输出一个 JSON 对象，其中包含以下键：French_summary，num_names。

请用换行符分隔您的答案。

Text:
```{text}```
"""
response = get_completion(prompt_1)
print("prompt 1:")
print(response)

prompt 1:
1-兄妹在山顶井里打水时发生意外，但仍然保持冒险精神。
2-Dans un charmant village, les frère et sœur Jack et Jill partent chercher de l'eau dans un puits au sommet de la montagne. Malheureusement, Jack trébuche sur une pierre et tombe de la montagne, suivi de près par Jill. Bien qu'ils soient légèrement blessés, ils retournent chez eux chaleureusement. Malgré cet accident, leur esprit d'aventure ne diminue pas et ils continuent à explorer joyeusement.
3-Jack, Jill
4-{
   "French_summary": "Dans un charmant village, les frère et sœur Jack et Jill partent chercher de l'eau dans un puits au sommet de la montagne. Malheureusement, Jack trébuche sur une pierre et tombe de la montagne, suivi de près par Jill. Bien qu'ils soient légèrement blessés, ils retournent chez eux chaleureusement. Malgré cet accident, leur esprit d'aventure ne diminue pas et ils continuent à explorer joyeusement.",
   "num_names": 2
}


上述输出仍然存在一定问题，例如，键“姓名”会被替换为法语，因此，我们给出一个更好的 Prompt，该 Prompt 指定了输出的格式

In [ ]:
prompt_2 = f"""
Your task is to perform the following actions: 
1 - Summarize the following text delimited by <> with 1 sentence.
2 - Translate the summary into French.
3 - List each name in the French summary.
4 - Output a json object that contains the 
following keys: french_summary, num_names.

Use the following format:
Text: <text to summarize>
Summary: <summary>
Translation: <summary translation>
Names: <list of names in French summary>
Output JSON: <json with summary and num_names>

Text: <{text}>
"""
response = get_completion(prompt_2)
print("\nCompletion for prompt 2:")
print(response)


Completion for prompt 2:
Summary: 兄妹杰克和吉尔在山顶井里打水时发生意外，但他们仍然保持冒险精神继续探索。
Translation: Jack and Jill, deux frères et sœurs, ont eu un accident en allant chercher de l'eau dans un puits de montagne, mais ils ont continué à explorer avec un esprit d'aventure.
Names: Jack, Jill
Output JSON: {"french_summary": "Jack and Jill, deux frères et sœurs, ont eu un accident en allant chercher de l'eau dans un puits de montagne, mais ils ont continué à explorer avec un esprit d'aventure.", "num_names": 2}


In [ ]:
prompt_2 = f"""
1-用一句话概括下面用<>括起来的文本。
2-将摘要翻译成英语。
3-在英语摘要中列出每个名称。
4-输出一个 JSON 对象，其中包含以下键：English_summary，num_names。

请使用以下格式：
文本：<要总结的文本>
摘要：<摘要>
翻译：<摘要的翻译>
名称：<英语摘要中的名称列表>
输出 JSON：<带有 English_summary 和 num_names 的 JSON>

Text: <{text}>
"""
response = get_completion(prompt_2)
print("\nprompt 2:")
print(response)


prompt 2:
摘要：兄妹杰克和吉尔在迷人的村庄里冒险，不幸摔伤后回到家中，但仍然充满冒险精神。
翻译：In a charming village, siblings Jack and Jill set out to fetch water from a mountaintop well. While climbing and singing, Jack trips on a stone and tumbles down the mountain, with Jill following closely behind. Despite some bruises, they make it back home safely. Their adventurous spirit remains undiminished as they continue to explore with joy.
名称：Jack，Jill
输出 JSON：{"English_summary": "In a charming village, siblings Jack and Jill set out to fetch water from a mountaintop well. While climbing and singing, Jack trips on a stone and tumbles down the mountain, with Jill following closely behind. Despite some bruises, they make it back home safely. Their adventurous spirit remains undiminished as they continue to explore with joy.", "num_names": 2}


**策略二：指导模型在下结论之前找出一个自己的解法**

有时候，在明确指导模型在做决策之前要思考解决方案时，我们会得到更好的结果。

接下来我们会给出一个问题和一个学生的解答，要求模型判断解答是否正确

In [ ]:
prompt = f"""
Determine if the student's solution is correct or not.

Question:
I'm building a solar power installation and I need \
 help working out the financials. 
- Land costs $100 / square foot
- I can buy solar panels for $250 / square foot
- I negotiated a contract for maintenance that will cost \ 
me a flat $100k per year, and an additional $10 / square \
foot
What is the total cost for the first year of operations 
as a function of the number of square feet.

Student's Solution:
Let x be the size of the installation in square feet.
Costs:
1. Land cost: 100x
2. Solar panel cost: 250x
3. Maintenance cost: 100,000 + 100x
学生的解决方案：
设x为发电站的大小，单位为平方英尺。
费用：
Total cost: 100x + 250x + 100,000 + 100x = 450x + 100,000
"""
response = get_completion(prompt)
print(response)

The student's solution is correct.


In [ ]:
prompt = f"""
判断学生的解决方案是否正确。

问题:
我正在建造一个太阳能发电站，需要帮助计算财务。

    土地费用为 100美元/平方英尺
    我可以以 250美元/平方英尺的价格购买太阳能电池板
    我已经谈判好了维护合同，每年需要支付固定的10万美元，并额外支付每平方英尺10美元
    作为平方英尺数的函数，首年运营的总费用是多少。

学生的解决方案：
设x为发电站的大小，单位为平方英尺。
费用：

    土地费用：100x
    太阳能电池板费用：250x
    维护费用：100,000美元+100x
    总费用：100x+250x+100,000美元+100x=450x+100,000美元
"""
response = get_completion(prompt)
print(response)

学生的解决方案是正确的。


但是注意，学生的解决方案实际上是错误的。

我们可以通过指导模型先自行找出一个解法来解决这个问题。

在接下来这个 Prompt 中，我们要求模型先自行解决这个问题，再根据自己的解法与学生的解法进行对比，从而判断学生的解法是否正确。同时，我们给定了输出的格式要求。通过明确步骤，让模型有更多时间思考，有时可以获得更准确的结果。在这个例子中，学生的答案是错误的，但如果我们没有先让模型自己计算，那么可能会被误导以为学生是正确的。

In [ ]:
prompt = f"""
Your task is to determine if the student's solution \
is correct or not.
To solve the problem do the following:
- First, work out your own solution to the problem. 
- Then compare your solution to the student's solution \ 
and evaluate if the student's solution is correct or not. 
Don't decide if the student's solution is correct until 
you have done the problem yourself.

Use the following format:
Question:
```
question here
```
Student's solution:
```
student's solution here
```
Actual solution:
```
steps to work out the solution and your solution here
```
Is the student's solution the same as actual solution \
just calculated:
```
yes or no
```
Student grade:
```
correct or incorrect
```

Question:
```
I'm building a solar power installation and I need help \
working out the financials. 
- Land costs $100 / square foot
- I can buy solar panels for $250 / square foot
- I negotiated a contract for maintenance that will cost \
me a flat $100k per year, and an additional $10 / square \
foot
What is the total cost for the first year of operations \
as a function of the number of square feet.
``` 
Student's solution:
```
Let x be the size of the installation in square feet.
Costs:
1. Land cost: 100x
2. Solar panel cost: 250x
3. Maintenance cost: 100,000 + 100x
Total cost: 100x + 250x + 100,000 + 100x = 450x + 100,000
```
Actual solution:
"""
response = get_completion(prompt)
print(response)

Let x be the size of the installation in square feet.

Costs:
1. Land cost: 100x
2. Solar panel cost: 250x
3. Maintenance cost: 100,000 + 10x

Total cost: 100x + 250x + 100,000 + 10x = 360x + 100,000

Is the student's solution the same as actual solution just calculated:
No

Student grade:
Incorrect


In [ ]:
prompt = f"""
请判断学生的解决方案是否正确，请通过如下步骤解决这个问题：

步骤：

    首先，自己解决问题。
    然后将你的解决方案与学生的解决方案进行比较，并评估学生的解决方案是否正确。在自己完成问题之前，请勿决定学生的解决方案是否正确。

使用以下格式：

    问题：问题文本
    学生的解决方案：学生的解决方案文本
    实际解决方案和步骤：实际解决方案和步骤文本
    学生的解决方案和实际解决方案是否相同：是或否
    学生的成绩：正确或不正确

问题：

    我正在建造一个太阳能发电站，需要帮助计算财务。 
    - 土地费用为每平方英尺100美元
    - 我可以以每平方英尺250美元的价格购买太阳能电池板
    - 我已经谈判好了维护合同，每年需要支付固定的10万美元，并额外支付每平方英尺10美元
    作为平方英尺数的函数，首年运营的总费用是多少。

学生的解决方案：

    设x为发电站的大小，单位为平方英尺。
    费用：
    1. 土地费用：100x
    2. 太阳能电池板费用：250x
    3. 维护费用：100,000+100x
    总费用：100x+250x+100,000+100x=450x+100,000

实际解决方案和步骤：
"""
response = get_completion(prompt)
print(response)

正确的解决方案和步骤：
    1. 计算土地费用：100美元/平方英尺 * x平方英尺 = 100x美元
    2. 计算太阳能电池板费用：250美元/平方英尺 * x平方英尺 = 250x美元
    3. 计算维护费用：10万美元 + 10美元/平方英尺 * x平方英尺 = 10万美元 + 10x美元
    4. 计算总费用：100x美元 + 250x美元 + 10万美元 + 10x美元 = 360x + 10万美元

学生的解决方案和实际解决方案是否相同：否

学生的成绩：不正确


## 三、局限性

**虚假知识**：模型偶尔会生成一些看似真实实则编造的知识

如果模型在训练过程中接触了大量的知识，它并没有完全记住所见的信息，因此它并不很清楚自己知识的边界。这意味着它可能会尝试回答有关晦涩主题的问题，并编造听起来合理但实际上并不正确的答案。我们称这些编造的想法为幻觉。

例如在如下示例中，我们要求告诉我们 Boie 公司生产的 AeroGlide UltraSlim Smart Toothbrush 产品的信息，事实上，这个公司是真实存在的，但产品是编造的，模型则会一本正经地告诉我们编造的知识。



In [ ]:
prompt = f"""
Tell me about AeroGlide UltraSlim Smart Toothbrush by Boie
"""
response = get_completion(prompt)
print(response)

The AeroGlide UltraSlim Smart Toothbrush by Boie is a high-tech toothbrush that uses advanced sonic technology to provide a deep and thorough clean. It features a slim and sleek design that makes it easy to hold and maneuver, and it comes with a range of smart features that help you optimize your brushing routine.

One of the key features of the AeroGlide UltraSlim Smart Toothbrush is its advanced sonic technology, which uses high-frequency vibrations to break up plaque and bacteria on your teeth and gums. This technology is highly effective at removing even the toughest stains and buildup, leaving your teeth feeling clean and fresh.

In addition to its sonic technology, the AeroGlide UltraSlim Smart Toothbrush also comes with a range of smart features that help you optimize your brushing routine. These include a built-in timer that ensures you brush for the recommended two minutes, as well as a pressure sensor that alerts you if you're brushing too hard.

Overall, the AeroGlide UltraS

In [ ]:
prompt = f"""
告诉我 Boie 公司生产的 AeroGlide UltraSlim Smart Toothbrush 的相关信息
"""
response = get_completion(prompt)
print(response)

Boie公司生产的AeroGlide UltraSlim Smart Toothbrush是一款智能牙刷，具有以下特点：

1. 超薄设计：刷头仅有0.8毫米的厚度，可以更容易地进入口腔深处，清洁更彻底。

2. 智能感应：牙刷配备了智能感应技术，可以自动识别刷头的位置和方向，确保每个部位都得到充分的清洁。

3. 高效清洁：牙刷采用了高速振动技术，每分钟可达到40000次，可以有效去除牙菌斑和污渍。

4. 轻松携带：牙刷采用了便携式设计，可以轻松放入口袋或旅行包中，随时随地进行口腔清洁。

5. 环保材料：牙刷采用了环保材料制造，不含有害物质，对环境友好。

总之，Boie公司生产的AeroGlide UltraSlim Smart Toothbrush是一款高效、智能、环保的牙刷，可以帮助用户轻松保持口腔健康。


模型会输出看上去非常真实的编造知识，这有时会很危险。因此，请确保使用我们在本节中介绍的一些技巧，以尝试在构建自己的应用程序时避免这种情况。这是模型已知的一个弱点，也是我们正在积极努力解决的问题。在你希望模型根据文本生成答案的情况下，另一种减少幻觉的策略是先要求模型找到文本中的任何相关引用，然后要求它使用这些引用来回答问题，这种追溯源文档的方法通常对减少幻觉非常有帮助。

**说明：在本教程中，我们使用 \ 来使文本适应屏幕大小以提高阅读体验，GPT 并不受 \ 的影响，但在你调用其他大模型时，需额外考虑 \ 是否会影响模型性能**